In [16]:
import pandas as pd
from autogluon.tabular import TabularPredictor

# 加载数据
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv') # 如果这行代码仅用于加载测试数据，而不是立即使用，可以暂时注释掉

# 定义预测目标列名
label = 'Rating'

# 删除目标列中的空值行
train_data = train_data.dropna(subset=[label])

# 指定使用MLP模型
hyperparameters = {
    'NN_TORCH': {},  # 使用PyTorch神经网络模型，并使用默认配置
}

# AutoGluon训练，仅使用MLP模型
predictor = TabularPredictor(label=label, eval_metric='root_mean_squared_error', problem_type='regression').fit(
    train_data,
    presets='good_quality',
    hyperparameters=hyperparameters
)


No path specified. Models will be saved in: "AutogluonModels/ag-20240320_025157"
Presets specified: ['good_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240320_025157/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.


In [ ]:
# 预测测试集
predictions = predictor.predict(test_data)

# 把预测结果作为新列添加到测试数据的副本中
test_data_with_predictions = test_data.copy()
test_data_with_predictions[label] = predictions

# 创建Keys列，直接将userId和productId组合成一个元组
test_data_with_predictions['Keys'] = list(zip(test_data_with_predictions['userId'], test_data_with_predictions['productId']))

# 仅保留Keys和预测结果列
submission = test_data_with_predictions[['Keys', label]]

# 保存预测结果到CSV文件，以符合提交格式
submission.to_csv('submission.csv', index=False)